In [1]:
import cv2
import numpy as np

# Load YOLOv3 model and configuration
net = cv2.dnn.readNet("E:/AI ML internship BEL/yolov3-tiny.weights", "E:/AI ML internship BEL/yolov3-tiny (1).cfg")

# Load COCO class names (or your custom class names)
with open("E:/AI ML internship BEL/coco.names", "r") as f:
    classes = f.read().strip().split("\n")

# Create a list to store object centroids and their corresponding classes
object_centroids = []

# Function to calculate the Euclidean distance between two points
def calculate_distance(point1, point2):
    return np.sqrt((point1[0] - point2[0])**2 + (point1[1] - point2[1])**2)

# Define the Region of Interest (ROI) coordinates (x, y, width, height)
roi_x, roi_y, roi_width, roi_height = 299, 300, 300, 500

# Open a video capture source (0 for webcam, or provide a video file path)
cap = cv2.VideoCapture(0)  # Change to your video source if needed

while True:
    ret, frame = cap.read()

    if not ret:
        break

    # Crop the frame to the ROI
    roi = frame[roi_y:roi_y + roi_height, roi_x:roi_x + roi_width]

    # Get frame dimensions and prepare input blob for YOLO
    height, width = frame.shape[:2]
    blob = cv2.dnn.blobFromImage(frame, 1/255.0, (416, 416), swapRB=True, crop=False)

    # Set the input blob for YOLO
    net.setInput(blob)

    # Get layer names
    layer_names = net.getUnconnectedOutLayersNames()

    # Run forward pass to get YOLO detections
    detections = net.forward(layer_names)

    # Reset the list of object centroids for this frame
    object_centroids = []

    # Process YOLO detections
    for detection in detections:
        for obj in detection:
            scores = obj[5:]
            class_id = np.argmax(scores)
            confidence = scores[class_id]

            if confidence > 0.2:
                # Extract bounding box coordinates
                box = obj[0:4] * np.array([width, height, width, height])
                (x, y, w, h) = box.astype("int")

                # Calculate centroid
                centroid_x = x + w / 2
                centroid_y = y + h / 2

                # Check if the centroid is within the ROI
                if roi_x < centroid_x < roi_x + roi_width and roi_y < centroid_y < roi_y + roi_height:
                    # Draw bounding box and label
                    color = (0, 255, 0)  # Green color
                    cv2.rectangle(frame, (x, y), (x + w, y + h), color, 2)
                    text = f"{classes[class_id]}: {confidence:.2f}"
                    cv2.putText(frame, text, (x, y - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)

                    # Store object centroid and class
                    object_centroids.append((centroid_x, centroid_y, classes[class_id]))

    # Display the ROI on the frame
    cv2.rectangle(frame, (roi_x, roi_y), (roi_x + roi_width, roi_y + roi_height), (0, 0, 255), 2)

    # Display the result
    cv2.imshow("Object Detection", frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()
